In [7]:
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the TensorFlow model from the .h5 file
MODEL_PATH = "/kaggle/input/bertmodel/tensorflow2/default/1/multi_task_bert_model.h5"
custom_objects = {'TFBertModel': TFBertModel}  # Register custom layer
with tf.keras.utils.custom_object_scope(custom_objects):
    model = tf.keras.models.load_model(MODEL_PATH)

# Define a function for inference
def predict_sentiment(text):
    """
    Predicts the sentiment and rank score for a given input text.
    """
    # Tokenize the input text
    inputs = tokenizer(
        text, 
        max_length=128, 
        truncation=True, 
        padding='max_length', 
        return_tensors='tf',
        return_token_type_ids=False  # Disable token_type_ids
    )
    
    # Perform inference
    outputs = model([inputs['input_ids'], inputs['attention_mask']])
    
    # Inspect the outputs structure
    print("Outputs:", outputs)
    
    # Extract the sentiment score (regression output)
    sentiment_score = outputs[0].numpy()[0][0]  # Assuming sentiment_score is the first output
    
    # Map the sentiment score to a label
    sentiment_label = "POSITIVE" if sentiment_score > 0.5 else "NEGATIVE"
    
    # Get the rank score (if your model outputs it)
    if len(outputs) > 1:  # Assuming rank_score is the second element in outputs
        rank_score = outputs[1].numpy()[0][0]
    else:
        rank_score = None
    
    return {
        "text": text,
        "sentiment": sentiment_label,
        "sentiment_score": float(sentiment_score),
        "rank_score": float(rank_score) if rank_score is not None else None
    }

# Test the model with some example text
example_text = "The company reported strong earnings and a record high stock price."
result = predict_sentiment(example_text)

# Print the result
print("Input Text:", result["text"])
print("Sentiment:", result["sentiment"])
print("Sentiment Score:", result["sentiment_score"])
print("Rank Score:", result["rank_score"])

Outputs: [<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.97613424]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.7709417]], dtype=float32)>]
Input Text: The company reported strong earnings and a record high stock price.
Sentiment: POSITIVE
Sentiment Score: 0.9761342406272888
Rank Score: 4.770941734313965
